# Options Pricing with Binomial Model

*Goals:*

* Value OMXS30 options with Binomial Model and continuous dividends
* Value American options with Discrete Dividends
* Compute the implied volatility of American options using the Binomial Model

*Data:*

* Risk Free Interest Rate Curve (OIS)
* Futures prices to back out continous dividends
* Dividends, Maturities, and other Contract Specifications

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

%config InlineBackend.figure_format = 'retina'
sns.set_theme()
sns.set_style("whitegrid", {'grid.linestyle': '--'})
seq_col_brew = sns.color_palette("flag_r", 4)
sns.set_palette(seq_col_brew)

* Options Price (bid/ask): 55.50/59.50
* Option Vol (bid/ask): 22.85/24.61
* OMXS30 Points (close): 1965.29 
* OIS discount factor: 0.9984 (1M) 24 days
* Futures Price: 1964.50 @ 18 NOV
* Time To Maturity (years): 24/365
* Time Step: 1/365


In [6]:
rate = -1/(1/12) * np.log(0.9984)

In [7]:
rate

0.01921537640368655

In [58]:
from scipy.stats import norm

def black76(imp_vol, forward, strike, expiry, rate, type="C"):
    d1 = 1/(imp_vol * np.sqrt(expiry)) * (np.log(forward/strike) + imp_vol**2/2 * expiry)
    d2 = d1 - imp_vol * np.sqrt(expiry)
    discount = np.exp(-rate * expiry)
    return discount * (forward * norm.cdf(d1) - strike * norm.cdf(d2)) 

In [59]:
from scipy.optimize import root_scalar

# Extracting the implied volatility.
def formatted_function(imp_vol):
    return black76(imp_vol, 1964.50, 1950, 24/365, rate, "C")- 55.90
print(root_scalar(formatted_function, bracket=[0, 1]))
print(black76(0.5204576110995609, 1964.50, 1950, 24/365, rate, "C"))

      converged: True
           flag: 'converged'
 function_calls: 8
     iterations: 7
           root: 0.2416455921013494
111.40000000000018


/var/folders/d7/rpl_88f12ln2z1s2h4m9ybjm0000gn/T/ipykernel_6691/2957959625.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  d1 = 1/(imp_vol * np.sqrt(expiry)) * (np.log(forward/strike) + imp_vol**2/2 * expiry)


In [65]:
def price_option(imp_vol, spot, strike, expiry, delta_t, rate, div_yield, type="CE"):
    """
    type: Call European (CE), Put European (PE), Call American (CA), Put American (PA)
    """

    # Initialize parameters.
    u = np.exp(imp_vol*np.sqrt(delta_t))
    d = 1/u
    q = (np.exp((rate-div_yield) * delta_t) - d) / (u - d)
    steps = int(np.floor(expiry/delta_t))
    
    # Build spot matrix. TODO: Jagged arrays.
    spot_matrix = np.zeros((steps+1, steps+1))
    for i in range(steps+1):
        for j in range(steps+1):
            if j >= i:
                spot_matrix[i,j] = spot * u **(j-i) * d**(i)

    # Pricing EURPOEAN CALL OPTIONS HERE:


    # Pricing AMERICAN CALL OPTIONS HERE:


    option_price = 0
    
    return option_price

In [76]:
price_option(0.2, 100, 50, 8/250, 1/250, 0.01, 0.01)

[[100.         101.2729449  102.56209369 103.86765263 105.18983062
  106.52883921 107.88489264 109.25820788 110.64900467]
 [  0.          98.74305531 100.         101.2729449  102.56209369
  103.86765263 105.18983062 106.52883921 107.88489264]
 [  0.           0.          97.50190972  98.74305531 100.
  101.2729449  102.56209369 103.86765263 105.18983062]
 [  0.           0.           0.          96.27636465  97.50190972
   98.74305531 100.         101.2729449  102.56209369]
 [  0.           0.           0.           0.          95.06622399
   96.27636465  97.50190972  98.74305531 100.        ]
 [  0.           0.           0.           0.           0.
   93.87129414  95.06622399  96.27636465  97.50190972]
 [  0.           0.           0.           0.           0.
    0.          92.6913839   93.87129414  95.06622399]
 [  0.           0.           0.           0.           0.
    0.           0.          91.52630447  92.6913839 ]
 [  0.           0.           0.           0.           

0